<a href="https://colab.research.google.com/github/WongPowa/CUDABillinearInterpolation/blob/main/CUDAAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Header file for cv_pipe
%%writefile cv_pipe.h
#pragma once
#include <opencv2/opencv.hpp>
#include <iostream>
#include <unistd.h> // For pipe
#include <fcntl.h>  // For O_WRONLY

int cv_imshow(cv::Mat &image);
int init_cv_pipe_comm(int argc, char *argv[], bool verbose=false);
int finalize_cv_pipe_comm();

Writing cv_pipe.h


In [ ]:
%%writefile cv_pipe.cpp
#include "cv_pipe.h"

int fd = -1;

int open_named_pipe(char *pipe_name) {
    fd = open(pipe_name, O_WRONLY);
    if(fd < 0) {
        std::cerr << "Error: failed to open the named pipe: "
                  << pipe_name << std::endl;
    }
    return fd;
}

int cv_imshow(cv::Mat &image) {
    if(fd < 0) {
        std::cerr << "Error: no named pipe available." << std::endl;
        return -1;
    }
    // Send image size as a header
    int img_size[3] = {image.cols, image.rows, image.channels()};
    write(fd, img_size, sizeof(img_size));
    // Send the image data
    write(fd, image.data, image.total() * image.elemSize());
    return 0;
}

int init_cv_pipe_comm(int argc, char *argv[], bool verbose) {
    int c;
    char *pipe_path = NULL;

    if(verbose) {
        // Print all input arguments
        for(int i = 0; i < argc; i++) {
            std::cout << "[" << i << "] " << argv[i] << std::endl;
        }
    }
    //opterr = 0;       // Do not print error to stderr
    while ((c = getopt(argc, argv, ":p:")) != -1) {
        switch(c) {
            case 'p':
                pipe_path = optarg;
                break;
            case ':':
                std::cerr << "Error: option -" << static_cast<char>(optopt)
                          << " requires an argument.\n";
                return -1;
            case '?':
                // Ignore all unknown options; let the main program handles it.
                break;
        }
    }
    if(!pipe_path) {
        std::cerr << "Error: expect a pipe name but none found. Try the "
                  << "following:\n\t" << argv[0] << " -p my_pipe\n";
        return -1;
    }

    fd = open_named_pipe(pipe_path);
    return fd;
}

int finalize_cv_pipe_comm() {
    close(fd);        // Close the write end of the pipe
    return 0;
}

Writing cv_pipe.cpp


In [ ]:
%%writefile runner.py
import os, sys, subprocess
import threading
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

def tee_pipe(pipe, out):
    for line in pipe:
        #print(line.decode('utf-8'), end='')
        #print(line.decode('utf-8'), end='',  file=out)
        out.write(line.decode('utf-8'))

def execute(filename, *args, pipe_name='/tmp/my_pipe'):
    if not os.path.exists(pipe_name):
        os.mkfifo(pipe_name)

    # Start the subprocess. The -u option is to force the Python subprocess
    # to flush its output everytime it prints.
    proc = subprocess.Popen(
            [filename, '-p', pipe_name, *args],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
    )
    # Create threads to capture and print stdout and stderr
    t1 = threading.Thread(target=tee_pipe, args=(proc.stdout, sys.stdout))
    t2 = threading.Thread(target=tee_pipe, args=(proc.stderr, sys.stdout))
    t1.start()
    t2.start()

    with open(pipe_name, "rb") as pipe:
        while True:
            # Read the image size from the pipe
            # The 1st 4 byte is column size
            # The 2nd 4 byte is row size
            # The 3rd 4 byte is channel size
            img_header = pipe.read(12)
            if not img_header:
                break
            image_size = np.frombuffer(img_header, dtype=np.uint32)
            # Read the image data for all channels
            frame_data = pipe.read(image_size[0] * image_size[1] * image_size[2])
            if not frame_data:
                break
            frame = np.frombuffer(frame_data, dtype=np.uint8).reshape((image_size[1], image_size[0], image_size[2]))
            # Display the received frame
            cv2_imshow(frame)

    proc.wait()                 # Wait for subprocess to exit
    os.remove(pipe_name)        # Clean up the named pipe
    cv2.destroyAllWindows()

Writing runner.py


In [ ]:
%%writefile image_rescaler.cpp
#include <opencv2/opencv.hpp>
#include <vector>
#include <cmath>
#include <cuda_runtime.h>
#include <iostream>
#include "cv_pipe.h"

#define reset_getopt()    (optind = 0)
using namespace std;

vector<vector<vector<double>>> bilinear_resize_color(const vector<vector<vector<double>>>& image, int height, int width);
cv::Mat vectorToMatColor(const vector<vector<vector<double>>>& imageVector);

#ifndef __GPU_TIMER_H__
#define __GPU_TIMER_H__

struct GpuTimer
{
      cudaEvent_t start;
      cudaEvent_t stop;

      GpuTimer()
      {
            cudaEventCreate(&start);
            cudaEventCreate(&stop);
      }

      ~GpuTimer()
      {
            cudaEventDestroy(start);
            cudaEventDestroy(stop);
      }

      void Start()
      {
            cudaEventRecord(start, 0);
      }

      void Stop()
      {
            cudaEventRecord(stop, 0);
      }

      float Elapsed()
      {
            float elapsed;
            cudaEventSynchronize(stop);
            cudaEventElapsedTime(&elapsed, start, stop);
            return elapsed;
      }
};

#endif  /* __GPU_TIMER_H__ */

int main(int argc, char* argv[])
{
    int c;
    std::vector<char*> img_filenames;
    std::vector<vector<vector<double>>> image_vector;

     // (2) Initialize the cv_pipe
    init_cv_pipe_comm(argc, argv, true);

    // (3) Parse the option arguments from the user
    reset_getopt();
    while ((c = getopt(argc, argv, "p:")) != -1) {
        switch (c) {
            case 'p':
                // Do nothing because it should be handled by cv_pipe
                break;
            case '?':
                // Abort when encountering an unknown option
                return -1;
        }
    }
    // (4) Parse the non-option arguments from the user
    for (int index = optind; index < argc; index++)
        // Get all filenames if any and put them in the 'img_filenames' vector
        img_filenames.push_back(argv[index]);

    //for (int i = 1; i < argc; i++) {
    //    img_filenames.push_back(argv[i]);
    //}

    if (img_filenames.empty()) {
        std::cerr << "No image filenames provided." << std::endl;
        return -1;
    }

    GpuTimer totalTimer;
    totalTimer.Start(); // Start the timer

    for (auto filename : img_filenames) {
        std::cout << "Loading file: " << filename << std::endl;
        cv::Mat image = cv::imread(filename, cv::IMREAD_COLOR);
        if (image.empty()) {
            std::cerr << "Unable to load image: " << filename << std::endl;
            return -1;
        }

        // Convert image to 3D vector (for RGB channels)
        image_vector = std::vector<vector<vector<double>>>(image.rows, vector<vector<double>>(image.cols, vector<double>(3)));
        for (int i = 0; i < image.rows; ++i) {
            for (int j = 0; j < image.cols; ++j) {
                cv::Vec3b pixel = image.at<cv::Vec3b>(i, j);
                image_vector[i][j][0] = static_cast<double>(pixel[0]); // B channel
                image_vector[i][j][1] = static_cast<double>(pixel[1]); // G channel
                image_vector[i][j][2] = static_cast<double>(pixel[2]); // R channel
            }
        }

        int new_height = 512;
        int new_width = 512;

        GpuTimer timer;
        timer.Start(); // Start the timer

        vector<vector<vector<double>>> resized = bilinear_resize_color(image_vector, new_height, new_width);

        timer.Stop();
        printf("Time to generate:  %3.1f ms \n", timer.Elapsed());

        // Convert the resized vector back to an OpenCV Mat
        cv::Mat resized_image = vectorToMatColor(resized);

        // Display the resized image
        cv_imshow(resized_image);
    }

    return 0;
}

vector<vector<vector<double>>> bilinear_resize_color(const vector<vector<vector<double>>>& image, int height, int width) {
    int img_height = image.size();
    int img_width = image[0].size();

    vector<vector<vector<double>>> resized(height, vector<vector<double>>(width, vector<double>(3)));

    double x_ratio = (width > 1) ? static_cast<double>(img_width - 1) / (width - 1) : 0;
    double y_ratio = (height > 1) ? static_cast<double>(img_height - 1) / (height - 1) : 0;

    for (int i = 0; i < height; ++i) {
        for (int j = 0; j < width; ++j) {
            int x_l = floor(x_ratio * j);
            int y_l = floor(y_ratio * i);
            int x_h = ceil(x_ratio * j);
            int y_h = ceil(y_ratio * i);

            double x_weight = (x_ratio * j) - x_l;
            double y_weight = (y_ratio * i) - y_l;

            for (int c = 0; c < 3; ++c) {
                double a = image[y_l][x_l][c];
                double b = image[y_l][x_h][c];
                double c_val = image[y_h][x_l][c];
                double d = image[y_h][x_h][c];

                resized[i][j][c] = a * (1 - x_weight) * (1 - y_weight) +
                    b * x_weight * (1 - y_weight) +
                    c_val * y_weight * (1 - x_weight) +
                    d * x_weight * y_weight;
            }
        }
    }

    return resized;
}

cv::Mat vectorToMatColor(const vector<vector<vector<double>>>& imageVector) {
    int rows = imageVector.size();
    int cols = imageVector[0].size();

    cv::Mat image(rows, cols, CV_8UC3);

    for (int i = 0; i < rows; ++i) {
        for (int j = 0; j < cols; ++j) {
            image.at<cv::Vec3b>(i, j)[0] = static_cast<uchar>(std::clamp(imageVector[i][j][0], 0.0, 255.0));  // B
            image.at<cv::Vec3b>(i, j)[1] = static_cast<uchar>(std::clamp(imageVector[i][j][1], 0.0, 255.0));  // G
            image.at<cv::Vec3b>(i, j)[2] = static_cast<uchar>(std::clamp(imageVector[i][j][2], 0.0, 255.0));  // R
        }
    }

    return image;
}


Writing image_rescaler.cpp


In [ ]:
!nvcc -o image_rescaler image_rescaler.cpp cv_pipe.cpp `pkg-config --cflags --libs opencv4`

In [ ]:
from runner import execute

execute(
    # Execute the program `image_rescaler` we compiled above
    "./image_rescaler",
    # Pass the 3 image filenames downloaded from the web
    "girlsun.jpg"
)

[0] ./image_rescaler
[1] -p
[2] /tmp/my_pipe
[3] girlsun.jpg
Loading file: girlsun.jpg
Unable to load image: girlsun.jpg


In [ ]:
%%writefile cuda_image_rescaler.cuh
#pragma once

#ifdef __CUDACC__
__global__ void bilinear_resize_color_kernel(const double* d_image, double* d_resized,
                                             int img_width, int img_height, int resized_width,
                                             int resized_height, double x_ratio, double y_ratio);
#endif

#include <vector>
#include <opencv2/opencv.hpp> // Include OpenCV

void bilinear_resize_core(const std::vector<std::vector<std::vector<double>>>& image,
                          int new_width, int new_height, int channels);

cv::Mat vectorToMatColor(const std::vector<std::vector<std::vector<double>>>& imageVector);


Writing cuda_image_rescaler.cuh


In [ ]:
%%writefile cuda_image_rescaler.cpp
#include <iostream>
#include <cuda_runtime.h>
#include "cuda_image_rescaler.cuh"
#include <opencv2/opencv.hpp>
#include <vector>
#include <cmath>
#include <iostream>
#include "cv_pipe.h"

#define reset_getopt()    (optind = 0)
using namespace std;

#ifndef __GPU_TIMER_H__
#define __GPU_TIMER_H__

struct GpuTimer
{
      cudaEvent_t start;
      cudaEvent_t stop;

      GpuTimer()
      {
            cudaEventCreate(&start);
            cudaEventCreate(&stop);
      }

      ~GpuTimer()
      {
            cudaEventDestroy(start);
            cudaEventDestroy(stop);
      }

      void Start()
      {
            cudaEventRecord(start, 0);
      }

      void Stop()
      {
            cudaEventRecord(stop, 0);
      }

      float Elapsed()
      {
            float elapsed;
            cudaEventSynchronize(stop);
            cudaEventElapsedTime(&elapsed, start, stop);
            return elapsed;
      }
};

#endif  /* __GPU_TIMER_H__ */

__global__ void bilinear_resize_color_kernel(const double* d_image, double* d_resized,
                                             int img_width, int img_height, int resized_width,
                                             int resized_height, double x_ratio, double y_ratio) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < resized_height && j < resized_width) {
        int x_l = floor(x_ratio * j);
        int y_l = floor(y_ratio * i);
        int x_h = min(x_l + 1, img_width - 1);
        int y_h = min(y_l + 1, img_height - 1);

        double x_weight = (x_ratio * j) - x_l;
        double y_weight = (y_ratio * i) - y_l;

        //for each color channel (R,G,B)
        for (int c = 0; c < 3; ++c) {
            double a = d_image[(y_l * img_width + x_l) * 3 + c]; //top-left
            double b = d_image[(y_l * img_width + x_h) * 3 + c]; //top-right
            double c_val = d_image[(y_h * img_width + x_l) * 3 + c]; //bottom-left
            double d = d_image[(y_h * img_width + x_h) * 3 + c]; //bottom-right

            d_resized[(i * resized_width + j) * 3 + c] = a * (1 - x_weight) * (1 - y_weight) +
                                                         b * x_weight * (1 - y_weight) +
                                                         c_val * y_weight * (1 - x_weight) +
                                                         d * x_weight * y_weight;
        }
    }
}

cv::Mat vectorToMatColor(const vector<vector<vector<double>>>& imageVector) {
    int rows = imageVector.size();
    int cols = imageVector[0].size();

    cv::Mat image(rows, cols, CV_8UC3);

    for (int i = 0; i < rows; ++i) {
        for (int j = 0; j < cols; ++j) {
            image.at<cv::Vec3b>(i, j)[0] = static_cast<uchar>(std::clamp(imageVector[i][j][0], 0.0, 255.0));  // B
            image.at<cv::Vec3b>(i, j)[1] = static_cast<uchar>(std::clamp(imageVector[i][j][1], 0.0, 255.0));  // G
            image.at<cv::Vec3b>(i, j)[2] = static_cast<uchar>(std::clamp(imageVector[i][j][2], 0.0, 255.0));  // R
        }
    }

    return image;
}

const int thread_per_blk = 32;

void bilinear_resize_core(const std::vector<std::vector<std::vector<double>>>& image,
                            int new_width, int new_height, int channels) {
  int img_height = image.size();
  int img_width = image[0].size();

  size_t img_size = img_width * img_height * 3 * sizeof(double);
  size_t resized_size = new_width * new_height * 3 * sizeof(double);

  //Host pointers
  double* h_image = new double[img_width * img_height * 3];
  double* h_resized = new double[new_width * new_height * 3];

  // Flatten the 3D image vector to 1D array
    for (int i = 0; i < img_height; ++i) {
        for (int j = 0; j < img_width; ++j) {
            for (int c = 0; c < 3; ++c) {
                h_image[(i * img_width + j) * 3 + c] = image[i][j][c];
            }
        }
    }

  //Device pointers
  double* d_image, * d_resized;

  cudaMalloc(&d_image, img_size);
  cudaMalloc(&d_resized, img_size);

  // Copy the original image data to device memory
  cudaMemcpy(d_image, h_image, img_size, cudaMemcpyHostToDevice);

  // Compute the x_ratio and y_ratio
  double x_ratio = static_cast<double>(img_width - 1) / (new_width - 1);
  double y_ratio = static_cast<double>(img_height - 1) / (new_height - 1);

  // Define the block and grid size
  dim3 block(thread_per_blk, thread_per_blk);
  dim3 grid((new_width + block.x - 1) / block.x, (new_height + block.y - 1) / block.y);

  GpuTimer timer;
  timer.Start(); // Start the timer

  // Launch the kernel
  bilinear_resize_color_kernel<<<grid, block>>>(d_image, d_resized, img_width, img_height, new_width, new_height, x_ratio, y_ratio);

  timer.Stop();
  printf("Time to generate:  %3.1f ms \n", timer.Elapsed());

  // Copy the resized image data back to the host
  cudaMemcpy(h_resized, d_resized, resized_size, cudaMemcpyDeviceToHost);

  // Convert the resized vector back to an OpenCV Mat
  std::vector<std::vector<std::vector<double>>> resized_vector(new_height, std::vector<std::vector<double>>(new_width, std::vector<double>(3)));

  // Convert 1D h_resized back to 3D vector
  for (int i = 0; i < new_height; ++i) {
      for (int j = 0; j < new_width; ++j) {
          for (int c = 0; c < 3; ++c) {
              resized_vector[i][j][c] = h_resized[(i * new_width + j) * 3 + c];
          }
      }
  }

  cv::Mat resized_image = vectorToMatColor(resized_vector);

  // Display the resized image
  cv_imshow(resized_image);

  // Free device memory
  cudaFree(d_image);
  cudaFree(d_resized);
  delete[] h_image;
  delete[] h_resized;
}

int main(int argc, char* argv[])
{
    int c;
    std::vector<char*> img_filenames;
    std::vector<vector<vector<double>>> image_vector;

     // (2) Initialize the cv_pipe
    init_cv_pipe_comm(argc, argv, true);

    // (3) Parse the option arguments from the user
    reset_getopt();
    while ((c = getopt(argc, argv, "p:")) != -1) {
        switch (c) {
            case 'p':
                // Do nothing because it should be handled by cv_pipe
                break;
            case '?':
                // Abort when encountering an unknown option
                return -1;
        }
    }
    // (4) Parse the non-option arguments from the user
    for (int index = optind; index < argc; index++)
        // Get all filenames if any and put them in the 'img_filenames' vector
        img_filenames.push_back(argv[index]);

    if (img_filenames.empty()) {
        std::cerr << "No image filenames provided." << std::endl;
        return -1;
    }

    for (auto filename : img_filenames) {
        std::cout << "Loading file: " << filename << std::endl;
        cv::Mat image = cv::imread(filename, cv::IMREAD_COLOR);
        if (image.empty()) {
            std::cerr << "Unable to load image: " << filename << std::endl;
            return -1;
        }

        // Convert image to 3D vector (for RGB channels)
        image_vector = std::vector<vector<vector<double>>>(image.rows, vector<vector<double>>(image.cols, vector<double>(3)));
        for (int i = 0; i < image.rows; ++i) {
            for (int j = 0; j < image.cols; ++j) {
                cv::Vec3b pixel = image.at<cv::Vec3b>(i, j);
                image_vector[i][j][0] = static_cast<double>(pixel[0]); // B channel
                image_vector[i][j][1] = static_cast<double>(pixel[1]); // G channel
                image_vector[i][j][2] = static_cast<double>(pixel[2]); // R channel
            }
        }
    }

    int new_height = 512;
    int new_width = 512;

    bilinear_resize_core(image_vector, new_width, new_height, 3);

    return 0;
}

Writing cuda_image_rescaler.cpp


In [ ]:
!mv cuda_image_rescaler.cpp cuda_image_rescaler.cu
!nvcc -o cuda_image_rescaler cuda_image_rescaler.cu cv_pipe.cpp `pkg-config --cflags --libs opencv4`

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [ ]:
from runner import execute

execute(
    # Execute the program `cuda_image_rescaler` we compiled above
    "./cuda_image_rescaler",
    # Pass the 3 image filenames downloaded from the web
    "girlsun.jpg"
)

[0] ./cuda_image_rescaler
[1] -p
[2] /tmp/my_pipe
[3] girlsun.jpg
Loading file: girlsun.jpg
Unable to load image: girlsun.jpg
